In [32]:
import requests, os, json, re, ezodf
import pandas as pd
from pathlib import Path
# from datetime import date, timedelta, datetime
baseurl="https://ecp.mis66.ru"
cookie_file=Path("cookies.txt")
users=[{'name':'Admin','username':os.getenv('ECP_USERNAME_1'), 'password':os.getenv('ECP_PASSWORD_1')},
      {'name':'User','username':os.getenv('ECP_USERNAME_4'), 'password':os.getenv('ECP_PASSWORD_4'),'LpuSection':'99560017798','MedStaffFact':'62646','EMDCertificate_id':'22132'}]
current_user=users[0]

def login(username,password):
    print("Logining")
    headers = {
            "Login": username,
            "Password": password
    }
    response=session.request("GET", baseurl+"/api/user/login", headers=headers).json()
    print(response)

def login(username,password):
    print("Logining")
    headers = {
            "Login": username,
            "Password": password
    }
    response=session.request("GET", baseurl+"/api/user/login", headers=headers).json()
    print(response)

def load_directions(date_from, date_to, surname=''):
    payload = {'EvnDirection_IsAuto' : 0,
               'EvnQueueStatus_id' : 1,
               'Lpu_did' : '',#'13003619',
               # 'MedPersonal_did': 1596,
               'object' : 'EvnDirection',
               'beg_date' : date_from,
               'end_date' : date_to,
               'RecordDate_from':date_from,
                'RecordDate_to':date_to,
                'limit':1000,
                'start':0,
                'LpuSectionProfile_did':'',#'401048',
                'EvnStatus_id':'',#'10,17',
                'useCase':'',#'open_from_polka',
                'winType':'queue',
                'SearchType':'',#'incoming',
               'dateRangeMode':'allTime',
               'onlyWaitingList':'true',
                'Person_SurName': surname,
                }
    response = session.request("POST", baseurl+"/?c=EvnDirection&m=loadBaseJournal", params=payload).json()
    print(response)
    return response

def cancel_direction(queue_id, direction_id):
    payload = {'cancelType' : 'decline',
               'EvnQueue_id' : queue_id,
               'EvnStatusCause_id' : 1,
               'EvnDirection_id' : direction_id,
               'DirType_Code' : 3,
                }
    response = session.request("POST", baseurl+"/?c=Queue&m=cancelQueueRecord", params=payload)#.json()
    return response
    
def get_person_by_id(person_id):
    url = baseurl+"/api/Person"
    response = session.request("GET", url, params={"Person_id": person_id})
    return response.json()

def get_dead_patients(date_from, date_to):
    url=baseurl+'/?c=Search&m=searchData'
    payload = {'PersonPeriodicType_id':1,
                'SearchFormType':'EvnPS',
                'EvnPS_InRegistry':0,
                'Okei_id':100,
                'EvnPS_disDate_Range':f'{date_from} - {date_to}',
                'Date_Type':1,
                'LeaveType_id':143,
                'SearchType_id':1,
                'PersonCardStateType_id':1,
                # 'PrivilegeStateType_id':1,
                'limit':9000,
                'start':0,
                }
    response = session.request("POST", url, params=payload).json()
    print(f'найдено трупов {len(response['data'])}, за период с {date_from} по {date_to}')
    return response['data']

#---------------------------------------- Найти первое из свиделтьств о смерти пациента через PromedWeb
def get_person_svid_list(person_id, user_doctor_id=156131):
    try:
        url=baseurl+'/?c=Template&m=getEvnFormPersonSvidInfo'
        payload = {'user_MedStaffFact_id':user_doctor_id,
                   'Evn_id':person_id,
                   'object':'PersonSvidInfo',
                   'object_id':'Person_id',
                   'object_value':person_id,
                   'archiveRecord':'0',
                   'ARMType':'common',
                   'from_MZ':1,
                   'from_MSE':1,
                   'parent_object_value':person_id,
                   'parent_object_id':'Person_id',
                   'Person_id':person_id,
                  }
        response = session.request("GET", url, params=payload).json()
        return response
    except Exception as e:
        print(f"Error sending request: {e}")
        return None

#---------------------------------------- Найти свиделтьство о смерти пациента
def get_person_svid_id(person_id, user_doctor_id=156131):
    data = get_person_svid_list(person_id)
    try:
        # Check if the expected structure exists
        if "map" in data and "PersonSvidInfo" in data["map"]:
            person_info = data["map"]["PersonSvidInfo"]
            
            # Check if item list exists and has at least one entry
            if "item" in person_info and len(person_info["item"]) > 0:
                # Get the first item's data
                print(f'нашёл {len(person_info["item"])} свидетельств[о/а]')
                first_item = person_info["item"][0]
                
                # Extract PersonSvid_id from the data
                if "data" in first_item and "PersonSvid_id" in first_item["data"]:
                    return first_item["data"]["PersonSvid_id"]
        
        # If we couldn't find it through the expected path, try a regex approach as backup
        match = re.search(r'"PersonSvid_id":"([^"]+)"', response_text)
        if match:
            return match.group(1)
            
        return None
    except Exception as e:
        print(f"Error parsing response: {e}")
        return None
    
#-------------------------------------- Получить свидетельство о смерти пациента через PromedWeb (ошибка в api)
def get_ecp_death_svid_by_person_id(person_id): # lpu_id=ГИБ
    url=baseurl+'/?c=MedSvid&m=loadMedSvidEditForm'
    death_svid_id=get_person_svid_id(person_id)
    payload = {'svid_id':death_svid_id,
               'svid_type':'death',
              }
    response = session.request("GET", url, params=payload).json()[0]
    return response

#------------------------------------------ Convert a list of dictionaries to an ODS file.
def list_of_dicts_to_ods(data_list, output_file):
    try:
        if output_file[-4:] != '.ods':
            print('output_file[-4:] =', output_file[-4:], 'appending .ods')
            output_file += '.ods'

        if not isinstance(data_list, list) or not all(isinstance(item, dict) for item in data_list):
            raise ValueError("Input must be a list of dictionaries")
        
        # Function to sanitize strings for XML compatibility
        def sanitize_for_xml(value):
            if value is None:
                return ""
            
            # Convert to string if not already
            if not isinstance(value, str):
                if isinstance(value, (dict, list)):
                    value = json.dumps(value)
                else:
                    value = str(value)
            
            # Remove XML-incompatible characters
            # XML 1.0 allows only these characters:
            # #x9 | #xA | #xD | [#x20-#xD7FF] | [#xE000-#xFFFD] | [#x10000-#x10FFFF]
            illegal_xml_chars = re.compile(
                u'[\x00-\x08\x0B\x0C\x0E-\x1F\uD800-\uDFFF\uFFFE\uFFFF]'
            )
            return illegal_xml_chars.sub('', value)
        
        # Sanitize all data in the list of dictionaries
        sanitized_data = []
        for item in data_list:
            sanitized_item = {}
            for key, value in item.items():
                sanitized_key = sanitize_for_xml(key)
                sanitized_value = sanitize_for_xml(value)
                sanitized_item[sanitized_key] = sanitized_value
            sanitized_data.append(sanitized_item)
        
        # Convert sanitized list of dictionaries to DataFrame
        df = pd.DataFrame(sanitized_data)
        
        # Create a new ODS file
        ods_doc = ezodf.newdoc(doctype="ods", filename=output_file)
        
        # Create a new sheet with appropriate dimensions
        sheet = ezodf.Sheet('Sheet1', size=(len(sanitized_data) + 1, len(df.columns)))
        ods_doc.sheets += sheet
        
        # Write column headers
        for col_idx, col_name in enumerate(df.columns):
            sheet[0, col_idx].set_value(str(col_name))
        
        # Write data rows
        for row_idx, row in df.iterrows():
            for col_idx, col_name in enumerate(df.columns):
                # Get the value (already sanitized)
                value = row[col_name]
                sheet[row_idx + 1, col_idx].set_value(value)
        
        # Save the file
        ods_doc.save()
        print(f"Successfully saved to {output_file}")
        return True
    
    except Exception as e:
        print(f"Error converting list of dictionaries to ODS: {e}")
        return False

session = requests.session()
login(current_user['username'], current_user['password'])

Logining
{'error_code': 0, 'sess_id': '2e6qg90dftm6k0kkqh7aa9j90m'}


In [6]:
r=cancel_direction(660101178000759,660101178000760)

print(r.json())

{'Error_Code': None, 'Error_Msg': None, 'success': True}


In [3]:
###---------отменить все направления со статусом "в очереди"
date_from='01.01.2025'
date_to='30.12.2025'
surname='КОЗЛОВА'#<-------------------------------------------------------------ФАМИЛИЮ СЮДА

load_directions_resp=load_directions(date_from, date_to,surname)
count=0 
for direction in load_directions_resp['data']:
    count+=1
    queue_id=direction['EvnQueue_id']
    direction_id=direction['EvnDirection_id']
    print(count, 'queue_id=',queue_id,'direction_id=',direction_id)
    print(cancel_direction(queue_id,direction_id))

{'data': [{'Person_id': '722883', 'TimeTableType_id': None, 'LpuUnit_did': '99560003957', 'MedPersonalDid': 'МАЛАХОВ АНАТОЛИЙ ВАСИЛЬЕВИЧ / Врач-инфекционист', 'MedStaffFact_IsQueueOnFree': '0', 'EvnStatus_SysNick': 'Queued', 'EvnDirection_RecDate': None, 'EvnDirection_setDate': '26.08.2025', 'EvnQueue_id': '660101242638819', 'EvnQueueStatus_Name': 'В очереди', 'QueueFailCause_Name': None, 'EvnQueueStatus_id': '1', 'MedService_id': None, 'TimetableMedService_id': None, 'EvnQueue_DeclineCount': None, 'EvnQueue_ServiceDT': '15.09.2025', 'TimetableResource_id': None, 'TimetableGraf_id': None, 'TimetableStac_id': None, 'EvnDirection_id': '660101242638820', 'EvnDirection_IsAuto': '1', 'EvnDirection_Num': '133920019', 'Lpu_sid': '13003668', 'LpuSectionProfile_id': '401048', 'LpuSectionProfile_Name': 'инфекционным болезням', 'recSort': None, 'Lpu_did': '13003619', 'DirType_id': '3', 'DirType_Code': '3', 'DirType_Name': 'На консультацию', 'MedStaffFact_id': '52518', 'Diag_Name': 'B18.2 Хроничес

In [8]:
#-------------------------------------- Найти свидетельство о смерти пациента через api (ошибка в api)
def get_death_svids(person_id, lpu_id=13003619): # lpu_id=ГИБ
    url=baseurl+'/api/DeathSvidList'
    payload = {'Person_id':person_id,
               'Lpu_id':lpu_id,
               'deathsvid_insdt':'',
              }
    response = session.request("GET", url, params=payload).json()
    return response

print(get_death_svids(285401))


{'error_code': 6, 'error_msg': 'Ошибка БД в файле core/SwModel.php (db: p_promed_ecp_api, host: 10.3.217.20):Error Number: \r\nERROR:  column bs.person_cid does not exist\nLINE 21:                  and ((bs.Person_id = \'285401\' OR bs.Person_...\n                                                           ^\r\n            select\n                bs.DeathSvid_id as "DeathSvid_id",\n                bs.DeathSvid_Ser as "DeathSvid_Ser",\n                bs.DeathSvid_Num as "DeathSvid_Num",\n                to_char(bs.DeathSvid_DeathDate, \'YYYY-MM-DD\') as "DeathSvid_DeathDate_Date",\n                to_char(bs.DeathSvid_DeathDate, \'hh24:mi\') as "DeathSvid_DeathDate_Time",\n                case when bs.DeathSvid_DeathDate is null then 2 else 1 end as "DeathSvid_IsUnknownDeathDate",\n                case when bs.DeathSvid_DeathDate is null then 2 else 1 end as "DeathSvid_isUnknownDeathTime",\n                dt.DeathSvidType_id as "DeathSvidType_id",\n                case when bs.DeathSvi

In [34]:
#-------------------------------------- Найти свидетельство о смерти пациента через promed
date_from='01.08.2025'
date_to='31.12.2025'
deads=get_dead_patients(date_from, date_to)
death_svids=[]
print(len(deads))
for d in deads:
    try:
        print(d['EvnPS_NumCard'], d['LeaveType_Name'],'Person_id =',d['Person_id'])
        death_svid=get_ecp_death_svid_by_person_id(d['Person_id'])
        death_svid['NumCard']=d['EvnPS_NumCard']
        death_svids.append(death_svid)
        print(len(death_svids))
    except Exception as e:
        print(str(e))
list_of_dicts_to_ods(death_svids,'death_svids')

найдено трупов 116, за период с 01.08.2025 по 31.12.2025
116
7131 Умер Person_id = 361434
нашёл 1 свидетельств[о/а]
1
7961 Умер Person_id = 660910001276820
нашёл 1 свидетельств[о/а]
2
8051 Умер Person_id = 703373
нашёл 1 свидетельств[о/а]
3
8053 Умер Person_id = 585002
нашёл 1 свидетельств[о/а]
4
8200 Умер Person_id = 88499
нашёл 1 свидетельств[о/а]
5
8319 Умер Person_id = 300974
нашёл 1 свидетельств[о/а]
6
8357 Умер Person_id = 304213
нашёл 1 свидетельств[о/а]
7
8453 Умер Person_id = 606737
нашёл 1 свидетельств[о/а]
8
8466 Умер Person_id = 268153
нашёл 1 свидетельств[о/а]
9
8468 Умер Person_id = 208122
нашёл 1 свидетельств[о/а]
10
8476 Умер Person_id = 1810250
нашёл 1 свидетельств[о/а]
11
8505 Умер Person_id = 350052
нашёл 1 свидетельств[о/а]
12
8527 Умер Person_id = 418859
нашёл 1 свидетельств[о/а]
13
8538 Умер Person_id = 877551
нашёл 1 свидетельств[о/а]
14
8546 Умер Person_id = 262212
нашёл 1 свидетельств[о/а]
15
8608 Умер Person_id = 213258
нашёл 1 свидетельств[о/а]
16
8653 Умер P

False

In [26]:
def get_dead_patients(date_from, date_to):
    url=baseurl+'/?c=Search&m=searchData'
    payload = {'PersonPeriodicType_id':1,#
                'SearchFormType':'EvnPS',#
                'EvnPS_InRegistry':0,#
                'Okei_id':100,#
                'EvnPS_disDate_Range':f'{date_from} - {date_to}',#
                'Date_Type':1,#
                'Ksg_Year':2025,#
                'LeaveType_id':143,#
                'SearchType_id':1,#
                'PersonCardStateType_id':1,#
                # 'PrivilegeStateType_id':1,
                'limit':9000,#
                'start':0,#
                }
    response = session.request("POST", url, params=payload).json()
    print(f'найдено трупов {len(response['data'])}, за период с {date_from} по {date_to}')
    return response['data']

deads=get_dead_patients(date_from, date_to)

найдено трупов 80, за период с 01.09.2025 по 31.12.2025


In [27]:
deads

[{'EvnPS_id': '660101230284147',
  'Person_id': '660910000620818',
  'PersonEvn_id': '106328521',
  'Server_id': '13003710',
  'EvnPS_IsTransit': '1',
  'EvnPS_NumCard': '8808',
  'EvnPS_setDate': '09.08.2025',
  'EvnPS_disDate': '11.09.2025',
  'LpuSection_Name': 'Инфекционное №3',
  'LpuSectionProfile_Name': 'инфекционным болезням',
  'Diag_Name': 'B20.8. Болезнь, вызванная ВИЧ, с проявлениями других инфекционных и паразитарных болезней',
  'EvnPS_KoikoDni': '33',
  'Person_IsBDZ': 'false',
  'PayType_Name': 'Местный бюджет',
  'LeaveType_Name': 'Умер',
  'LeaveType_Code': '105',
  'DeadSvid': 'true',
  'PrehospWaifRefuseCause_id': None,
  'EvnSection_KSG': ' ',
  'EvnSection_KSGKPG': ' ',
  'EvnSection_KPG': None,
  'EvnCostPrint_setDT': None,
  'EvnCostPrint_IsNoPrintText': '',
  'Person_Surname': 'РОБАКИДЗЕ',
  'Person_Firname': 'ВАХТАНГ',
  'Person_Secname': 'АВТАНДИЛОВИЧ',
  'Person_Birthday': '26.12.1980',
  'Person_deadDT': '11.09.2025',
  'fedservice_iemk': 'false',
  'EvnUsl

In [4]:
# Простой скрипт для преобразования
def simple_convert():
    # Читаем исходный файл
    with open('RefbookDiag.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Сохраняем с отключением ASCII преобразования
    with open('diagnoses.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    
    print("Преобразование завершено!")

# Запуск
simple_convert()

Преобразование завершено!


In [4]:
death_svids

[]

In [5]:
import pandas as pd
import pyexcel_ods3

url=baseurl+"/api/RefbookList"
payload = {}
response = session.request("GET", url, params=payload).json()

df = pd.DataFrame(response['data'])

# Create the data structure for pyexcel_ods3
# The correct format is a dictionary where keys are sheet names and values are 2D arrays
# First row should be headers, followed by data rows
sheet_data = []
# Add headers first
sheet_data.append(list(df.columns))
# Add data rows
for _, row in df.iterrows():
    sheet_data.append(list(row.values))

# Create the data dictionary with sheet name as key
data = {"Refbooks": sheet_data}

# Save to ODS file
output_file = "refbook_data.ods"
pyexcel_ods3.save_data(output_file, data)

print(f"Data successfully saved to {output_file}")

Data successfully saved to refbook_data.ods


In [15]:
url=baseurl+"/api/MedWorkerById"
payload = {'MedWorker_id':13950}
response = session.request("GET", url, params=payload).json()
print(response)

{'error_code': 0, 'data': {'Person_id': '624048', 'CodeDLO': '5629', 'HonouredBrevetDate': None, 'PeoplesBrevetDate': None}}


In [11]:
def get_lpu_by_id(lpu_id):
    url=baseurl+"/api/Refbook/Lpu"
    payload = {'Lpu_id':lpu_id}
    response = session.request("GET", url, params=payload).json()
    return response['data'][0]
lpu_id=13003797
lpu=get_lpu_by_id(lpu_id)
print(lpu['Lpu_f003mcod'])

660903


In [25]:
# GET api/MedWorkerById





{'Person_id': '816523', 'PersonSurName_SurName': 'САЛОМАТИНА', 'PersonFirName_FirName': 'ЕЛЕНА', 'PersonSecName_SecName': 'ВИКТОРОВНА', 'PersonBirthDay_BirthDay': '1977-04-18', 'Person_Sex_id': '2', 'PersonPhone_Phone': '9222262078', 'PersonSnils_Snils': '05321326007', 'PersonInn_Inn': '660703487858', 'SocStatus_id': '10000071', 'SocStatusFed_Code': '5', 'Person_pid': None, 'ParPersonSurName_SurName': None, 'ParPersonFirName_FirName': None, 'ParPersonSecName_SecName': None, 'DeputyKind_id': None, 'DeputyOrg_id': '0', 'DocumentAuthority_id': None, 'DocumentDeputy_Ser': None, 'DocumentDeputy_Num': None, 'DocumentDeputy_Issue': None, 'DocumentDeputy_begDate': None, 'UAddress_id': '11466062', 'UAddress_Address': '624790, РОССИЯ, СВЕРДЛОВСКАЯ ОБЛ, СВОБОДНЫЙ ПГТ, ЛЕНИНА УЛ, д. 42, кв. 42', 'PAddress_id': '11466063', 'PAddress_Address': '624790, РОССИЯ, СВЕРДЛОВСКАЯ ОБЛ, СВОБОДНЫЙ ПГТ, ЛЕНИНА УЛ, д. 42, кв. 42', 'BAddress_id': None, 'Org_id': '68320147111', 'Post_id': None, 'BDZ_guid': None, 

In [19]:

def get_person_by_id(person_id):
    url=baseurl+"/api/Person"
    payload = {'Person_id':person_id}
    response = session.request("GET", url, params=payload).json()
    return response['data']
print(get_person_by_id(624048))


[{'Person_id': '624048', 'PersonSurName_SurName': 'ДОРОХИНА', 'PersonFirName_FirName': 'ТАТЬЯНА', 'PersonSecName_SecName': 'ВЯЧЕСЛАВОВНА', 'PersonBirthDay_BirthDay': '1982-03-20', 'Person_Sex_id': '2', 'PersonPhone_Phone': '9506471550', 'PersonSnils_Snils': '21229244630', 'PersonInn_Inn': '662340451658', 'SocStatus_id': '10000071', 'SocStatusFed_Code': '5', 'Person_pid': None, 'ParPersonSurName_SurName': None, 'ParPersonFirName_FirName': None, 'ParPersonSecName_SecName': None, 'DeputyKind_id': None, 'DeputyOrg_id': '0', 'DocumentAuthority_id': None, 'DocumentDeputy_Ser': None, 'DocumentDeputy_Num': None, 'DocumentDeputy_Issue': None, 'DocumentDeputy_begDate': None, 'UAddress_id': '22842808', 'UAddress_Address': 'РОССИЯ, СВЕРДЛОВСКАЯ ОБЛ, Г НИЖНИЙ ТАГИЛ, АЛТАЙСКАЯ УЛ, д. 41\\26, кв. 96', 'PAddress_id': '22842809', 'PAddress_Address': 'РОССИЯ, СВЕРДЛОВСКАЯ ОБЛ, Г НИЖНИЙ ТАГИЛ, АЛТАЙСКАЯ УЛ, д. 41\\26, кв. 96', 'BAddress_id': None, 'Org_id': '68320144595', 'Post_id': None, 'BDZ_guid': Non

In [2]:
import os, re, time
from docx import Document
import subprocess
import tempfile
import json
import platform
import antiword
from pathlib import Path


def convert_docx_to_text(file_path):
    """Convert a .docx file to plain text"""
    try:
        import docx
        doc = docx.Document(file_path)
        text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
        return text
    except ImportError:
        raise FileNotFoundError("python-docx package is required for .docx conversion")
    except Exception as e:
        raise Exception(f"Error converting .docx file: {e}")

def convert_doc_to_text(file_path):
    """Convert a .doc file to plain text preserving Cyrillic characters"""
    try:
        # Try using pywin32 with Microsoft Word (best for Cyrillic)
        try:
            import win32com.client
            import os
            from pathlib import Path
            
            # Get absolute path
            abs_path = str(Path(file_path).resolve())
            
            # Create Word application
            word = win32com.client.Dispatch("Word.Application")
            word.visible = False
            
            # Open the document
            doc = word.Documents.Open(abs_path)
            
            # Extract text directly from the document
            text = ""
            for paragraph in doc.Paragraphs:
                text += paragraph.Range.Text + "\n"
            
            # Close Word
            doc.Close(False)  # Don't save changes
            word.Quit()
            
            return text
            
        except ImportError:
            # Fall back to docx2txt with encoding handling
            try:
                import subprocess
                import tempfile
                import os
                
                # Create a temporary docx file
                temp_docx = tempfile.NamedTemporaryFile(suffix='.docx', delete=False).name
                
                # Use LibreOffice or MS Word to convert doc to docx
                try:
                    # Try with LibreOffice first (if installed)
                    subprocess.run(['soffice', '--headless', '--convert-to', 'docx', 
                                   '--outdir', os.path.dirname(temp_docx), file_path], 
                                   check=True, capture_output=True)
                except (subprocess.SubprocessError, FileNotFoundError):
                    # If LibreOffice fails, try with MS Word via pywin32
                    import win32com.client
                    word = win32com.client.Dispatch("Word.Application")
                    word.visible = False
                    doc = word.Documents.Open(file_path)
                    doc.SaveAs2(temp_docx, FileFormat=16)  # 16 = docx
                    doc.Close()
                    word.Quit()
                
                # Now process the docx file
                import docx
                doc = docx.Document(temp_docx)
                text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
                
                # Clean up
                if os.path.exists(temp_docx):
                    os.remove(temp_docx)
                
                return text
                
            except Exception as e:
                # Last resort: try textract but warn about potential encoding issues
                import textract
                print("Warning: Using textract for .doc conversion. Cyrillic characters may not display correctly.")
                text = textract.process(file_path).decode('utf-8', errors='replace')
                return text
    except Exception as e:
        raise Exception(f"Error converting .doc file: {e}")

def convert_odt_to_text(file_path):
    """Convert a .odt file to plain text"""
    try:
        import odf.opendocument
        from odf.text import P
        
        doc = odf.opendocument.load(file_path)
        paragraphs = doc.getElementsByType(P)
        text = "\n".join([p.plainText() for p in paragraphs])
        return text
    except ImportError:
        raise FileNotFoundError("odfpy package is required for .odt conversion")
    except Exception as e:
        raise Exception(f"Error converting .odt file: {e}")

def extract_patient_info(text):
    """Extract patient information from document text"""
    patient = {}
    
    # Convert to lowercase for case-insensitive matching
    text_lower = text.lower()
    
    # Extract the relevant section between "Посмертный эпикриз" and "Дополнительные сведения"
    start_pattern = r'посмертный\s+эпикриз'
    end_pattern = r'дополнительные\s+сведения'
    
    start_match = re.search(start_pattern, text_lower)
    end_match = re.search(end_pattern, text_lower)
    
    if not start_match:
        return patient  # No "Посмертный эпикриз" found
    
    start_pos = start_match.start()
    end_pos = end_match.start() if end_match else len(text)
    
    relevant_text = text[start_pos:end_pos]
    print('------------------------------------------relevant_text =',relevant_text)
    # Extract patient name
    name_pattern1 = r'ФИО\s*[:]*\s*(\S+)\s+(\S+)\s+(\S+)'
    name_pattern2 = r'Фамилия\s*[:]*\s*(\S+)\s+Имя\s*[:]*\s*(\S+)\s+Отчество\s*[:]*\s*(\S+)'
    
    name_match = re.search(name_pattern1, relevant_text, re.IGNORECASE)
    if not name_match:
        name_match = re.search(name_pattern2, relevant_text, re.IGNORECASE)
    
    if name_match:
        patient['surname'] = name_match.group(1)
        patient['first_name'] = name_match.group(2)
        patient['patronymic'] = name_match.group(3)
    
    # Extract birth date
    birth_date_pattern = r'(?:дата\s+рождения|год\s+рождения|родился|родилась)\s*[:]*\s*(\d{1,2}\.\d{1,2}\.\d{2,4})'
    birth_match = re.search(birth_date_pattern, relevant_text, re.IGNORECASE)
    
    if birth_match:
        patient['birthdate'] = birth_match.group(1)
    
    # Extract death date
    death_date_pattern = r'дата\s+смерти\s*[:]*\s*(\d{1,2}\.\d{1,2}\.\d{2,4})'
    death_match = re.search(death_date_pattern, relevant_text, re.IGNORECASE)
    
    if death_match:
        patient['death_date'] = death_match.group(1)
    
    # Extract diagnosis
    diagnosis_start_pattern = r'(?:диагноз|основное\s+заболевание)\s*[:]*'
    diagnosis_start = re.search(diagnosis_start_pattern, relevant_text, re.IGNORECASE)
    
    if diagnosis_start:
        diagnosis_text = relevant_text[diagnosis_start.end():]
        # Keep the diagnosis text with its original formatting
        patient['diagnosis'] = diagnosis_text.strip()
        
        # Also provide an HTML-formatted version for better display
        html_diagnosis = diagnosis_text.strip()
        html_diagnosis = html_diagnosis.replace('\n', '<br>')
        html_diagnosis = html_diagnosis.replace('\t', '&nbsp;&nbsp;&nbsp;&nbsp;')
        html_diagnosis = html_diagnosis.replace('\r', '')
        patient['diagnosis_html'] = html_diagnosis
    
    return patient

def process_documents(directory):
    """Process all supported documents with proper format-specific handling"""
    results = []
    supported_extensions = ['.docx', '.doc', '.odt']
    
    # Check for required dependencies
    dependencies_ok = check_document_dependencies()
    if not dependencies_ok:
        print("Warning: Some document format dependencies are missing. Some files may fail.")
    
    for file_path in Path(directory).glob('**/*'):
        if file_path.suffix.lower() in supported_extensions:
            try:
                abs_path = str(file_path.absolute())
                print(f"Processing {abs_path}")
                
                # Use format-specific conversion based on file extension
                if file_path.suffix.lower() == '.docx':
                    text = convert_docx_to_text(abs_path)
                elif file_path.suffix.lower() == '.doc':
                    text = convert_doc_to_text(abs_path)
                elif file_path.suffix.lower() == '.odt':
                    text = convert_odt_to_text(abs_path)
                patient_info = extract_patient_info(text)
                
                if patient_info:
                    patient_info['source_file'] = abs_path
                    results.append(patient_info)
                else:
                    print(f"No patient information found in {file_path}")
            except FileNotFoundError as e:
                print(f"Dependency error processing {file_path}: {e}")
            except Exception as e:
                print(f"Error processing {file_path}: {e}")
    
    return results

def check_document_dependencies():
    """Check if all required dependencies for document conversion are available"""
    all_ok = True
    
    # Check for docx dependencies (usually pure Python)
    try:
        import docx
        print("✓ python-docx is available for .docx files")
    except ImportError:
        print("✗ python-docx is missing - .docx conversion may fail")
        all_ok = False
    
    # Check for .doc dependencies (usually requires LibreOffice or antiword)
    if platform.system() == 'Windows':
        # Check for LibreOffice on Windows
        libreoffice_paths = [
            r"C:\Program Files\LibreOffice\program\soffice.exe",
            r"C:\Program Files (x86)\LibreOffice\program\soffice.exe"
        ]
        if not any(os.path.exists(path) for path in libreoffice_paths):
            print("✗ LibreOffice not found - .doc and .odt conversion may fail")
            all_ok = False
        else:
            print("✓ LibreOffice found for .doc and .odt files")
    else:
        # On Linux/Mac check if commands are available
        try:
            subprocess.run(["which", "soffice"], check=True, stdout=subprocess.PIPE)
            print("✓ LibreOffice found for .doc and .odt files")
        except (subprocess.SubprocessError, FileNotFoundError):
            print("✗ LibreOffice not found - .doc and .odt conversion may fail")
            all_ok = False
    
    return all_ok
    
# Example usage
if __name__ == "__main__":
    directory = r"C:\Users\User\Anaconda\посмертные 2025"
    patients = process_documents(directory)
    
    # Save results to JSON file
    with open('patient_data.json', 'w', encoding='utf-8') as f:
        json.dump(patients, f, ensure_ascii=False, indent=4)
    
    print(f"Processed {len(patients)} documents with patient information")

✓ python-docx is available for .docx files
✓ LibreOffice found for .doc and .odt files
Processing C:\Users\User\Anaconda\посмертные 2025\Авдюкова Н.И.doc
------------------------------------------relevant_text = Посмертный эпикриз из истории болезни №2820
ФИО: АВДЮКОВА Н.И   пол: жен
Адрес регистрации:  Пригородный район Башкарка Новая 4- 2
Адрес фактического пребывания: тот же
Поступил: в стационар
Дата поступления: 10.03.25 Дата перевода в ОРиИТ:  10.03.25   Дата смерти: 11.03.2025
Количество дней нахождения в медицинской организации  1 дня
Форма   оказания   медицинской   помощи: экстренная


Основное: мезентериальный тромбоз. Перитонит.
Фоновый Ибс. акс
Осложнение: Внебольничная левосторонняя сегментарная пневмония. Хсн декомпенсация. 
Сопутствующие заболевания:  




Processing C:\Users\User\Anaconda\посмертные 2025\Агинских Ф.А. смерть.docx
------------------------------------------relevant_text = Посмертный эпикриз из истории болезни №5279
ФИО: Агинских Федор Алексеевич 28.05.20

In [19]:
temp=patients.copy()
for item in temp:
    # item.pop('diagnosis_html', None)
    item.pop('diagnosis', None)
list_of_dicts_to_ods(temp,'patients')

output_file[-4:] = ents appending .ods
Error converting list of dictionaries to ODS: [WinError 32] Процесс не может получить доступ к файлу, так как этот файл занят другим процессом: 'patients.ods' -> 'patients.ods.bak'


False

In [17]:
patients[:5]

[{'surname': 'АВДЮКОВА',
  'first_name': 'Н.И',
  'patronymic': 'пол:',
  'death_date': '11.03.2025',
  'source_file': 'C:\\Users\\User\\Anaconda\\посмертные 2025\\Авдюкова Н.И.doc'},
 {'surname': 'Агинских',
  'first_name': 'Федор',
  'patronymic': 'Алексеевич',
  'death_date': '29.04.24',
  'diagnosis_html': 'Вторичный гемофагоцитарный синдром. Острый лейкоз, неуточнённой этиологии. тяжелое течение. <br>Фоновый: Первичный иммунодефицит неуточнённый. Вторичная лимф-миелодисплазия кроветворения неуточнённой. Состояние после спленэктомии. Хроническая надпочечниковая недостаточность, медикаментозная компенсация. Периферическая моторно-сенсорная полинейропатия нижних конечностей, периферический нижний дистальный паралич, GMFCS 3-4 уровень. Синдром чувствительных нарушений по полинейропатическому типу в дистальных  отделах нижних конечностей. В анамнезе генерализованные приступы с судорогами тонико-клонического характера.      <br>Осложнение: Сепсис неуточнённой этиологии, септический шок,